In [ ]:
"""
# Install the transformers library if you haven't already
!pip install transformers

from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the BERT-Large tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

# Load the BERT-Large model for sequence classification (2 classes)
model = BertForSequenceClassification.from_pretrained('bert-large-uncased', num_labels=2)

# Example text
text = "gosdfasdfogle.com"

# Tokenize the input text
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted class
predictions = torch.argmax(outputs.logits, dim=-1)

print("Predicted class:", predictions.item())

"""

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted class: 0


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
ruta_archivo = '/content/drive/MyDrive/MOE_DGA/dga_dataset_16families.csv'  # Reemplaza con la ruta real
df = pd.read_csv(ruta_archivo)

print(df)

                                   domain     family  label
0                         hxejjordycd.com     nymaim    dga
1                         finishsafety.ru   suppobox    dga
2                           fgerntfdl.com     ramnit    dga
3       prostitutkivladivostokawithin.com      legit  legit
4                              aton-am.ru      legit  legit
...                                   ...        ...    ...
319995                   radyolojinet.com      legit  legit
319996                        zllwyswu.ws  conficker    dga
319997                          cgnet.com      legit  legit
319998                      suzuki.com.au      legit  legit
319999                      vfs-uk-ye.com      legit  legit

[320000 rows x 3 columns]


In [3]:
# Instalación de bibliotecas necesarias
%%capture
!pip install transformers datasets evaluate


In [4]:
# Importar bibliotecas
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from datasets import Dataset
import evaluate
import os

# Deshabilitar wandb
os.environ["WANDB_DISABLED"] = "true"


# Convertir las etiquetas de texto a numéricas
df['label_num'] = df['label'].map({'legit': 0, 'dga': 1})

# Dividir el dataframe en conjuntos de entrenamiento y prueba (80/20)
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

# Convertir los dataframes a datasets de HuggingFace y renombrar columnas
train_dataset = Dataset.from_pandas(train_df).rename_column('label_num', 'labels')
test_dataset = Dataset.from_pandas(test_df).rename_column('label_num', 'labels')

# Asegurar que las etiquetas sean enteros
def format_labels(example):
    example['labels'] = int(example['labels'])
    return example

train_dataset = train_dataset.map(format_labels)
test_dataset = test_dataset.map(format_labels)

# Cargar el tokenizador de ModernBERT
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")

# Función de tokenización para los dominios
def tokenize_function(examples):
    return tokenizer(
        examples["domain"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# Tokenizar los datasets y eliminar columnas innecesarias
tokenized_train = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['domain', 'label']
)

tokenized_test = test_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['domain', 'label']
)

# Cargar el modelo base
model = AutoModelForSequenceClassification.from_pretrained(
    "answerdotai/ModernBERT-base",
    num_labels=2
)

# Definir métricas de evaluación
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

# Definir los argumentos de entrenamiento
# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01
)

# Crear el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Entrenar el modelo
trainer.train()

# Evaluar el modelo
results = trainer.evaluate()
print(f"\n\nAccuracy final: {results['eval_accuracy']:.2%}")

# Guardar el modelo entrenado
trainer.save_model("./modernbert-dga-detector")

Map:   0%|          | 0/256000 [00:00<?, ? examples/s]

Map:   0%|          | 0/64000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

...

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-4-bcf065a28e39>:84: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
W0606 15:49:24.747000 466 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode


Step,Training Loss
500,0.226000
1000,0.160900
1500,0.136500
2000,0.131600
2500,0.127600
3000,0.113700
3500,0.114700


Step,Training Loss
500,0.226000
1000,0.160900
1500,0.136500
2000,0.131600
2500,0.127600
3000,0.113700
3500,0.114700
4000,0.107600
4500,0.075800
5000,0.074900




Accuracy final: 96.55%


In [5]:
# Crea una carpeta para tu modelo
save_path = "/content/drive/MyDrive/models/modernbert-dga-detector"

# Opcional: Crea el directorio si no existe
!mkdir -p {save_path}


# Usando el Trainer de Hugging Face
trainer.save_model(save_path)

# Opcional: Guardar también el tokenizer
trainer.tokenizer.save_pretrained(save_path)



Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('/content/drive/MyDrive/models/modernbert-dga-detector/tokenizer_config.json',
 '/content/drive/MyDrive/models/modernbert-dga-detector/special_tokens_map.json',
 '/content/drive/MyDrive/models/modernbert-dga-detector/tokenizer.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import requests
import pandas as pd
import numpy as np
import time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import sys
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
import re
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# CARGAR TU MODELO MODERNBERT
print("Cargando modelo ModernBERT...")
save_path = "/content/drive/MyDrive/models/modernbert-dga-detector"
model = AutoModelForSequenceClassification.from_pretrained(save_path)
tokenizer = AutoTokenizer.from_pretrained(save_path)

# Mover el modelo a GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print(f"Modelo cargado en: {device}")

# FUNCIÓN PARA PREDECIR CON TU MODELO
def predict_domain_modernbert(domain, model, tokenizer, device):
    """
    Predice si un dominio es DGA o no usando ModernBERT
    """
    # Tokenizar el dominio
    inputs = tokenizer(
        domain,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    # Mover inputs a GPU si está disponible
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Hacer predicción
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        prediction = torch.argmax(logits, dim=-1).item()

    # Convertir probabilidades a numpy
    probs = probabilities.cpu().numpy()[0]

    return {
        'prediction': prediction,  # 0 = notdga, 1 = dga
        'confidence': float(probs[prediction]),
        'prob_notdga': float(probs[0]),
        'prob_dga': float(probs[1])
    }



Cargando modelo ModernBERT...
Modelo cargado en: cuda


In [7]:
# CONFIGURACIÓN DE FAMILIAS Y RUNS
families = [
    'matsnu.gz',
    'suppobox.gz',
    'charbot.gz',
    'gozi.gz',
    'manuelita.gz',
    'rovnix.gz',
    'deception.gz',
    'nymaim.gz'
]

runs = 30

# BUCLE PRINCIPAL ADAPTADO
for family in families:
    print(f"\nProcesando familia: {family}")
    dga = pd.read_csv(f'/content/drive/My Drive/Familias_Test/{family}', chunksize=50)
    legit = pd.read_csv('/content/drive/My Drive/Familias_Test/legit.gz', chunksize=50)
    dfs = []

    for run in range(runs):
        print(f'{run:2}/{runs}', end='\r')
        dfw = pd.concat([dga.get_chunk(), legit.get_chunk()])
        pred = []
        prob_dga = []  # Probabilidades para DGA
        prob_notdga = []  # Probabilidades para not-DGA
        query_time = []

        for domain_to_check in dfw.domain.values:
            st = time.time()

            # USAR TU MODELO MODERNBERT EN LUGAR DEL GENERATIVO
            result = predict_domain_modernbert(domain_to_check, model, tokenizer, device)

            pred.append(result['prediction'])
            prob_dga.append(result['prob_dga'])
            prob_notdga.append(result['prob_notdga'])
            query_time.append(time.time() - st)

        # Agregar resultados al dataframe
        dfw['pred'] = pred
        dfw['prob_dga'] = prob_dga
        dfw['prob_notdga'] = prob_notdga
        dfw['query_time'] = query_time

        # Guardar resultados
        output_filename = f'/content/drive/My Drive/results/results_ModernBERT_DGA_{family}_{run}.csv.gz'
        dfw.to_csv(output_filename, index=False)



    print(f"\nCompletado procesamiento de {family}")

print("\n¡Evaluación completada!")


Procesando familia: matsnu.gz

Completado procesamiento de matsnu.gz

Procesando familia: suppobox.gz

Completado procesamiento de suppobox.gz

Procesando familia: charbot.gz

Completado procesamiento de charbot.gz

Procesando familia: gozi.gz

Completado procesamiento de gozi.gz

Procesando familia: manuelita.gz

Completado procesamiento de manuelita.gz

Procesando familia: rovnix.gz

Completado procesamiento de rovnix.gz

Procesando familia: deception.gz

Completado procesamiento de deception.gz

Procesando familia: nymaim.gz

Completado procesamiento de nymaim.gz

¡Evaluación completada!


In [9]:
# CONFIGURACIÓN DE FAMILIAS Y RUNS
families = ['bigviktor.gz',
            'pizd.gz',
            'ngioweb.gz'


           ]

runs = 30

# BUCLE PRINCIPAL ADAPTADO
for family in families:
    print(f"\nProcesando familia: {family}")
    dga = pd.read_csv(f'/content/drive/My Drive/New_Families/{family}', chunksize=50)
    legit = pd.read_csv('/content/drive/My Drive/Familias_Test/legit.gz', chunksize=50)
    dfs = []

    for run in range(runs):
        print(f'{run:2}/{runs}', end='\r')
        dfw = pd.concat([dga.get_chunk(), legit.get_chunk()])
        pred = []
        prob_dga = []  # Probabilidades para DGA
        prob_notdga = []  # Probabilidades para not-DGA
        query_time = []

        for domain_to_check in dfw.domain.values:
            st = time.time()

            # USAR TU MODELO MODERNBERT EN LUGAR DEL GENERATIVO
            result = predict_domain_modernbert(domain_to_check, model, tokenizer, device)

            pred.append(result['prediction'])
            prob_dga.append(result['prob_dga'])
            prob_notdga.append(result['prob_notdga'])
            query_time.append(time.time() - st)

        # Agregar resultados al dataframe
        dfw['pred'] = pred
        dfw['prob_dga'] = prob_dga
        dfw['prob_notdga'] = prob_notdga
        dfw['query_time'] = query_time

        # Guardar resultados
        output_filename = f'/content/drive/My Drive/results/results_ModernBERT_DGA_{family}_{run}.csv.gz'
        dfw.to_csv(output_filename, index=False)



    print(f"\nCompletado procesamiento de {family}")

print("\n¡Evaluación completada!")


Procesando familia: bigviktor.gz

Completado procesamiento de bigviktor.gz

Procesando familia: pizd.gz

Completado procesamiento de pizd.gz

Procesando familia: ngioweb.gz

Completado procesamiento de ngioweb.gz

¡Evaluación completada!


In [10]:
#"""
families = [
    'matsnu.gz',
    'suppobox.gz',
    'charbot.gz',
    'gozi.gz',
    'manuelita.gz',
    'rovnix.gz',
    'deception.gz',
    'nymaim.gz',
    'bigviktor.gz',
    'pizd.gz',
    'ngioweb.gz'
]
#"""

def fpr_tpr(y, ypred):
    tn, fp, fn, tp = confusion_matrix(y, ypred).ravel()
    fpr = fp / (fp + tn)  # False Positive Rate
    tpr = tp / (tp + fn)  # True Positive Rate (Recall)
    return fpr, tpr

for family in families:
    acc = []
    pre = []
    rec = []
    f1 = []
    fpr = []
    tpr = []
    qt = []
    qts = []
    for run in range(runs):
        df = pd.read_csv(f'/content/drive/My Drive/results/results_ModernBERT_DGA_{family}_{run}.csv.gz')
        y = (df.label == 'dga').astype(int)
        ypred = df.pred
        acc.append(accuracy_score(y, ypred))
        pre.append(precision_score(y, ypred))
        rec.append(recall_score(y, ypred))
        f1.append(f1_score(y, ypred))
        fpr_value, tpr_value = fpr_tpr(y, ypred)
        fpr.append(fpr_value)
        tpr.append(tpr_value)
        qt.append(df.query_time.mean())
        qts.append(df.query_time.std())
#    print(f'Query time: {np.mean(qt):0.5f}+/-{np.mean(qts)}:0.5f')
    print(f'{family.split(".")[0]:15}: acc:{np.mean(acc):0.2f}±{np.std(acc):.3f} f1:{np.mean(f1):0.2f}±{np.std(f1):.3f} pre:{np.mean(pre):0.2f}±{np.std(pre):.3f} rec:{np.mean(rec):0.2f}±{np.std(rec):.3f}  FPR:{np.mean(fpr):0.2f}±{np.std(fpr):.3f} TPR:{np.mean(tpr):0.2f}±{np.std(tpr):.3f} Query time: {np.mean(qt):0.5f}±{np.mean(qts):0.5f}')


matsnu         : acc:0.97±0.016 f1:0.97±0.016 pre:0.96±0.026 rec:0.99±0.019  FPR:0.04±0.028 TPR:0.99±0.019 Query time: 0.03500±0.02242
suppobox       : acc:0.98±0.016 f1:0.98±0.016 pre:0.96±0.026 rec:0.99±0.016  FPR:0.04±0.028 TPR:0.99±0.016 Query time: 0.03555±0.01012
charbot        : acc:0.89±0.032 f1:0.88±0.038 pre:0.96±0.031 rec:0.82±0.060  FPR:0.04±0.028 TPR:0.82±0.060 Query time: 0.02913±0.00690
gozi           : acc:0.90±0.031 f1:0.90±0.035 pre:0.96±0.029 rec:0.85±0.061  FPR:0.04±0.028 TPR:0.85±0.061 Query time: 0.02644±0.00315
manuelita      : acc:0.53±0.021 f1:0.16±0.060 pre:0.73±0.188 rec:0.09±0.037  FPR:0.04±0.028 TPR:0.09±0.037 Query time: 0.02675±0.00319
rovnix         : acc:0.98±0.015 f1:0.98±0.014 pre:0.96±0.026 rec:0.99±0.010  FPR:0.04±0.028 TPR:0.99±0.010 Query time: 0.02596±0.00308
deception      : acc:0.98±0.014 f1:0.98±0.013 pre:0.96±0.026 rec:1.00±0.000  FPR:0.04±0.028 TPR:1.00±0.000 Query time: 0.02581±0.00276
nymaim         : acc:0.91±0.026 f1:0.90±0.028 pre:0.96±